#THÔNG TIN SINH VIÊN

Họ Và Tên: Trương Minh Tuấn

MSSV: 19522485

Ngày thực hành: 28/09/2021


#THAO TÁC DỮ LIỆU


In [ ]:
!pip install matplotlib==3.1.3
!pip install osmnet
!pip install folium

!pip install rtree
!pip install pygeos
!pip install geojson
!pip install geopandas

     |████████████████████████████████| 13.1 MB 93 kB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 994 kB 6.9 MB/s 
     |████████████████████████████████| 6.3 MB 54.8 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 994 kB 5.1 MB/s 
     |████████████████████████████████| 1.8 MB 5.1 MB/s 


In [2]:
!git clone https://github.com/CityScope/CSL_HCMC

Cloning into 'CSL_HCMC'...
remote: Enumerating objects: 1930, done.
remote: Counting objects: 100% (590/590), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 1930 (delta 309), reused 500 (delta 270), pack-reused 1340
Receiving objects: 100% (1930/1930), 178.53 MiB | 25.80 MiB/s, done.
Resolving deltas: 100% (986/986), done.
Checking out files: 100% (542/542), done.


In [50]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import osmnet
import folium
import rtree
import pygeos
import geojson
import geopandas as gpd


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
file = gpd.read_file("/content/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp")

In [13]:
file.head(5)

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.40,24294.90,24292.90,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.60,38362.10,38704.40,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.10,43857.00,44204.00,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.50,35490.80,35594.20,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1..."


In [27]:
file.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [56]:
click_dt = gpd.read_file("/content/drive/MyDrive/CS116_B2/response3.json")

In [57]:
click_dt.head(5)

,dbh,geometry
0,1,POINT (106.71136 10.85113)
1,1,POINT (106.71695 10.74449)
2,1,POINT (106.66315 10.83289)
3,1,POINT (106.66267 10.83339)
4,1,POINT (106.65977 10.83729)


In [58]:
backup_crs = click_dt.crs
backup_crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [59]:
click_dt = gpd.read_file("/content/drive/MyDrive/CS116_B2/response3.json")

In [60]:
click_dt.head(5)

,dbh,geometry
0,1,POINT (106.71136 10.85113)
1,1,POINT (106.71695 10.74449)
2,1,POINT (106.66315 10.83289)
3,1,POINT (106.66267 10.83339)
4,1,POINT (106.65977 10.83729)


In [63]:
click_dt= click_dt.to_crs(epsg=32648)

In [64]:
click_dt.head(5)

,dbh,geometry
0,1,POINT (687074.226 1200044.747)
1,1,POINT (687752.360 1188252.815)
2,1,POINT (681813.990 1197998.313)
3,1,POINT (681761.573 1198052.679)
4,1,POINT (681442.508 1198483.069)


#YÊU CẦU BÀI TẬP:

TOP 3 QUẬN/HUYỆN CÓ TỐC ĐỘ TĂNG DÂN SỐ LỚN NHẤT

In [65]:
file["Tocdo"] = file["Pop_2019"]/file["Pop_2017"]

In [66]:
lst = list(file.sort_values("Tocdo", ascending=False)["Dist_Name"].head(10))
print("Top 10 quận/huyện có tốc độ tăng dân số lớn nhất là: ")
for i in lst:
  print("+ " + str(i))

Top 10 quận/huyện có tốc độ tăng dân số lớn nhất là: 
+ District 9
+ Hoc Mon
+ Nha Be
+ District 12
+ District 2
+ Cu Chi
+ District 7
+ Thu Duc
+ Binh Chanh
+ Binh Tan


LỌC CÁC ĐIỂM CLICK

In [67]:
filter_click = gpd.sjoin(click_dt, file, how="left", op="within")
filter_click.head(5)


,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,Tocdo
0,1,POINT (687074.226 1200044.747),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.20,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.9,52828.501430,4.775581e+07,1.092283
1,1,POINT (687752.360 1188252.815),4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,1.100641
2,1,POINT (681813.990 1197998.313),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07,1.020482
3,1,POINT (681761.573 1198052.679),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07,1.020482
4,1,POINT (681442.508 1198483.069),10,Go Vap,764,District,522690.0,548145.0,566216.0,588606.0,604143.0,620078.0,634146.0,646060.0,663313.0,676899.0,26510.80,27801.80,28718.40,29854.00,30642.00,31450.30,32163.80,32768.1,33643.10,34332.2,27550.300914,1.971611e+07,1.020482


In [68]:
filter_click_lst = filter_click[filter_click.Dist_Name.isin(lst)]
filter_click_lst.head()

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,Tocdo
0,1,POINT (687074.226 1200044.747),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.20,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.9,52828.50143,4.775581e+07,1.092283
1,1,POINT (687752.360 1188252.815),4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.91536,3.525220e+07,1.100641
5,1,POINT (691440.136 1200531.193),15,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.20,10487.80,10630.20,10842.20,11065.00,11245.9,11362.30,12410.9,52828.50143,4.775581e+07,1.092283
8,1,POINT (689464.643 1188780.763),4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.91536,3.525220e+07,1.100641
9,1,POINT (689796.719 1186376.959),4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.91536,3.525220e+07,1.100641


 CHẠY THUẬT TOÁN GOM NHÓM K-MEANS VỚI K = 20

In [86]:
#Convert
lst_point = []
for i in filter_click_lst["geometry"]:
  lst_point.append([i.x, i.y])

In [70]:
kmeans = KMeans(n_clusters=20, random_state=1).fit(lst_point)

In [71]:
labels = kmeans.labels_
print(labels)

[ 8 19  2 ... 19  2 11]


In [72]:
unique, counts = np.unique(labels, return_counts=True)
dict(zip(unique, counts))

{0: 2804,
 1: 2480,
 2: 3934,
 3: 61,
 4: 6152,
 5: 54,
 6: 590,
 7: 2601,
 8: 980,
 9: 4984,
 10: 57,
 11: 4474,
 12: 2588,
 13: 2737,
 14: 44,
 15: 18,
 16: 2823,
 17: 66,
 18: 3307,
 19: 4444}

LẤY RA CỤM ĐIỂM CLICK GOM NHÓM NHIỀU NHẤT CỦA MỖI QUẬN/HUYỆN

In [73]:
unique, counts = np.unique(labels, return_counts=True)
dic = dict(zip(unique, counts))
print(dic)

{0: 2804, 1: 2480, 2: 3934, 3: 61, 4: 6152, 5: 54, 6: 590, 7: 2601, 8: 980, 9: 4984, 10: 57, 11: 4474, 12: 2588, 13: 2737, 14: 44, 15: 18, 16: 2823, 17: 66, 18: 3307, 19: 4444}


In [81]:
max_label = max(dic, key=dic.get)
print(max_label)


4


In [82]:
len(labels)

45198

In [84]:
index_maxlabel = []
for i in range(len(labels)):
  if labels[i]==max_label:
    index_maxlabel.append(i) 
print(index_maxlabel)

[13, 14, 22, 30, 44, 46, 53, 91, 96, 101, 122, 128, 137, 139, 140, 142, 152, 167, 174, 183, 185, 195, 202, 218, 225, 228, 233, 235, 236, 238, 242, 248, 253, 270, 280, 284, 285, 287, 297, 300, 304, 305, 306, 307, 309, 315, 316, 317, 328, 329, 345, 348, 351, 353, 361, 384, 386, 387, 402, 426, 433, 438, 445, 450, 454, 456, 457, 459, 496, 513, 519, 527, 537, 545, 554, 555, 571, 579, 580, 584, 593, 596, 604, 606, 627, 633, 645, 652, 653, 670, 684, 704, 709, 717, 719, 720, 722, 743, 745, 748, 749, 769, 770, 774, 785, 786, 800, 806, 811, 812, 817, 819, 828, 844, 855, 870, 876, 888, 900, 905, 908, 909, 911, 933, 934, 943, 950, 951, 952, 954, 968, 978, 993, 994, 1000, 1002, 1011, 1033, 1047, 1049, 1060, 1061, 1062, 1071, 1072, 1077, 1097, 1100, 1119, 1121, 1123, 1124, 1136, 1138, 1141, 1144, 1157, 1167, 1169, 1171, 1172, 1183, 1195, 1197, 1209, 1217, 1218, 1224, 1225, 1231, 1237, 1246, 1247, 1249, 1264, 1270, 1273, 1274, 1303, 1304, 1313, 1322, 1328, 1330, 1334, 1345, 1346, 1361, 1363, 1368, 13

VẼ BIỂU ĐỒ HEATMAP

In [85]:
lst_geo = []
for i in index_maxlabel:
  lst_geo.append(filter_click_lst["geometry"])